<a href="https://colab.research.google.com/github/ShinAsakawa/xerion/blob/master/notebooks/2019wbaihandson_sequential_kmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# for [第2回失語症ハンズオン](https://wba-initiative.org/3411/)

<div>
    <center><img src="https://wba-initiative.org/wp-content/uploads/2015/05/logo.png" style="width:29%"></center>
</div>
    

<div align='right'>
<a href='mailto:asakawa@ieee.org'>Shin Aasakawa</a>, all rights reserved.<br>
Date: 15/Mar/2019<br>
 MIT license
</div>

---

## kmnist を系列処理とみなしてリカレントニューラルネットワークによる手書き文字認識

In [0]:
"""
[Original](mnist_hierarchical_rnn.py)

手書き文字データセット kmnist を系列処理とみなしてリカレントニューラルネットワークによる
系列処理で認識精度を実習する
"""
import numpy as np
import keras
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential

# Training parameters.
batch_size = 32
num_classes = 10
epochs = 5

Using TensorFlow backend.


In [0]:
# データの入手が未だの場合は以下の行頭の # を削除して本セルを実行してください
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz 
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz

--2019-03-15 04:51:32--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18384171 (18M)
Saving to: ‘kmnist-train-imgs.npz.1’

kmnist-train-imgs.n 100%[===================>]  17.53M  2.32MB/s    in 9.5s    

2019-03-15 04:51:42 (1.85 MB/s) - ‘kmnist-train-imgs.npz.1’ saved [18384171/18384171]

--2019-03-15 04:51:43--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29700 (29K)
Saving to: ‘kmnist-train-labels.npz.1’

kmnist-train-labels 100%[===================>]  29.00K  --.-KB/s    in 0.09s   

2019-03-15 04:51:43 (323 KB/s) - ‘kmnist-train-labels.npz.1’ saved [

In [0]:
# 直下行の N=10000 は実習の都合で実際のデータより少なくして学習時間を短くするためです
# 実際には全データ，すなわち N=60000 を使って評価する必要があります。
N=10000
X_train = np.load('kmnist-train-imgs.npz')['arr_0'][:N].reshape(-1,28,28)
X_test = np.load('kmnist-test-imgs.npz')['arr_0'].reshape(-1,28,28)
y_train = np.load('kmnist-train-labels.npz')['arr_0'][:N]
y_test = np.load('kmnist-test-labels.npz')['arr_0']
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((10000, 28, 28), (10000,), (10000, 28, 28), (10000,))

In [0]:
# 入出力を keras に入力できるように整形します
x_train = X_train.astype('float32')
x_test = X_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (10000, 28, 28)
10000 train samples
10000 test samples


In [0]:
model = Sequential()   # モデルの作成
model.add(LSTM(32, input_shape=(28, 28), return_sequences=True))
model.add(LSTM(16, return_sequences=False))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.summary()  # 作成したモデルほ表示

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 28, 32)            7808      
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                170       
Total params: 11,114
Trainable params: 11,114
Non-trainable params: 0
_________________________________________________________________


In [0]:
# 訓練の開始
epochs = 10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=True)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Instructions for updating:
Use tf.cast instead.
Train on 10000 samples, validate on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 13s 1ms/step - loss: 1.6217 - acc: 0.4614 - val_loss: 1.5657 - val_acc: 0.4752
Epoch 2/10
10000/10000 [==============================] - 11s 1ms/step - loss: 1.0996 - acc: 0.6664 - val_loss: 1.4222 - val_acc: 0.5463
Epoch 3/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.8839 - acc: 0.7301 - val_loss: 1.2291 - val_acc: 0.6075
Epoch 4/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.7445 - acc: 0.7745 - val_loss: 1.2039 - val_acc: 0.6197
Epoch 5/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.6434 - acc: 0.8059 - val_loss: 1.0739 - val_acc: 0.6618
Epoch 6/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.5771 - acc: 0.8203 - val_loss: 1.0191 - val_acc: 0.6865
Epoch 7/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0